<a href="https://colab.research.google.com/github/Lidiya-cutie/DS_SkillFactory/blob/main/Project_2_%D0%9A%D0%BE%D1%80%D0%BE%D1%82%D0%BA%D0%BE%D0%B2%D0%B0_%D0%9B%D0%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [1]:
import pandas as pd
import psycopg2

In [ ]:
# вставьте сюда параметры подключения из юнита 1. Работа с базой данных из Python 

In [2]:
connection = psycopg2.connect(
    dbname = 'project_sql',
    user = 'skillfactory',
    host = '84.201.134.129',
    password = 'cCkxxLVrDE8EbvjueeMedPKt',
    port = 5432
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [ ]:
# текст запроса
query_3_1 = f'''SELECT COUNT(*) cnt_vac
                FROM public.vacancies
          '''
df = pd.read_sql_query(query_3_1, connection)
df

In [ ]:
# результат запроса
|index|cnt_vac|
|-----|-------|
|  0  | 49197 |

2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [ ]:
# текст запроса
query_3_2 = f'''SELECT COUNT(*)
            FROM public.employers
         '''
df = pd.read_sql_query(query_3_2, connection)
df

In [ ]:
# результат запроса
|index|count|
|-----|-----|
|  0  |23501|

3. Посчитате с помощью запроса количество регионов (таблица areas).

In [ ]:
# текст запроса
query_3_3 = f'''SELECT COUNT(*)
            FROM public.areas
         '''
df = pd.read_sql_query(query_3_3, connection)
df

In [ ]:
# результат запроса
|index|count|
|-----|-----|
|  0  | 1362|

4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [ ]:
# текст запроса
query_3_4 = f'''SELECT COUNT(*) cnt_industries
            FROM public.industries
         '''
df = pd.read_sql_query(query_3_4, connection)
df

In [ ]:
# результат запроса
|index|cnt_industries|
|-----|--------------|
|  0  |     294      |

***

In [ ]:
# выводы по предварительному анализу данных

Итого:

Всего вакансии: 49197

Всего работодателей: 23501

Всего регионов(населенных пунктов): 1362

Всего сфер деятельности: 294

Предварительно, зная только эту информацию и учитывая связи между данными, можно определенно утвреждать, что:

* в среднем работодатель предлагает около 2-х вакансий (49197 / 23501)
* в среднем как минимум в одном в регионе предлагается примерно 36 вакансий (49197 / 1362)
* в среднем на одну сферу деятельности предлагается около 167 вакансий (49197 / 294)
* в среднем на один регион приходится по 17 работодателей (23501 / 1362)
* в среднем одна индустрия представлена 79 компаниями (работодателями) (23501 / 294)

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [ ]:
# текст запроса
n = 5
query_4_1 = f'''SELECT
                    a.name,
                    COUNT(v.id) cnt
                FROM public.VACANCIES v
                    JOIN public.employers e ON v.employer_id = e.id
                    JOIN public.areas a ON v.area_id = a.id
                GROUP BY a.name
                ORDER BY 2 DESC
                limit {n}
                '''


In [ ]:
df = pd.read_sql_query(query_4_1, connection)
df

In [ ]:
# результат запроса
|index|     name      |cnt |
|-----|---------------|----|
|  0  |   Москва      |5333|
|  1  |Санкт-Петербург|2851|
|  2  |    Минск      |2112|
|  3  |  Новосибирск  |2006|
|  4  |   Алматы      |1892|

2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [ ]:
# текст запроса
query_4_2 = f'''SELECT
                  COUNT(v.id) cnt
                FROM public.vacancies v
                WHERE salary_from IS NOT NULL OR salary_to IS NOT NULL
             '''
df = pd.read_sql_query(query_4_2, connection)
df

In [ ]:
# результат запроса
|index| cnt |
|-----|-----|
|  0  |24073|

3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [ ]:
# текст запроса
query_4_3 = f'''SELECT
                     ROUND(AVG(salary_from)) avg_salary_from,
                     ROUND(AVG(salary_to)) avg_salary_to
                 FROM public.vacancies v
             '''
df = pd.read_sql_query(query_4_3, connection)
df

<ipython-input-26-9940bf2a2926>:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_4_3, connection)


,avg_salary_from,avg_salary_to
0,71065.0,110537.0


In [ ]:
# результат запроса
|index|avg_salary_from|avg_salary_to|
|-----|---------------|-------------|
|  0  |    71065.0    |  110537.0   |

4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [ ]:
# текст запроса
query_4_4 = f'''SELECT
                  v.schedule,
                  v.employment,
                  COUNT(v.employment)
                FROM public.VACANCIES v
                GROUP BY  1, 2
                ORDER BY 3 DESC
                OFFSET 1 LIMIT 1
              '''
df = pd.read_sql_query(query_4_4, connection)
df

In [ ]:
# результат запроса
|index|    schedule    |   employment   |count|
|-----|----------------|----------------|-----|
|  0  |Удаленная работа|Полная занятость| 7802|

5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [ ]:
# текст запроса
query_4_5 = f'''SELECT 
                    DISTINCT(experience),
                    COUNT(id)
                FROM
                    public.vacancies v
                GROUP BY experience
                ORDER BY 2 ASC
                '''
df = pd.read_sql_query(query_4_5, connection)
df                

<ipython-input-11-09fdc416b0c6>:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_4_5, connection)


,experience,count
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


In [ ]:
# результат запроса
|index|   experience     |count|
|-----|------------------|-----|
| 0   |   более 6 лет    | 1337|
| 1   |   Нет опыта      | 7197|
| 2   |  От 3 до 6 лет   |14511|
| 3   |От 1 года до 3 лет|26152|

***

In [ ]:
# выводы по детальному анализу вакансий

* Можно заметить, что чем крупнее регион, тем вольше предлагается вакансий. Запрос на выведение числа (cnt) вакансий в каждом регионе (area) дал ожидаемый результат, так как в крупных городах вакансий ожидаемо больше, чем на периферии. 
Люди тянутся в большие города за более высоким и стабильным заработком. 
* Около половины вакансий содержат запись в одной из граф границ заработка, что говорит о том, что соискатели заинтересованы в определенном заработке, в остальных случаях, как вариант, зарплата устанавливается по результатам собеседования или же является фиксированной для определенных категорий граждан, например, бюджетников. 
* Средние значения для нижней и верхней границы зарплатной вилки показывают ожидаемые порганичные показатели заработной платы, которую готовы получать соикатели. При этом, средне-максимальная ЗП выше средне-минимальной в полтора раза: 110537 / 71065.
*  Запрос количества вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях показал, что наиболее актуальным для соискателей полный день с полной занятостью. Это классическая форма работы, которая дает соискателям уверенность "в завтрашнем дне". Именно поэтому она явный лидер, хотя на 'пятки уже наступает' вторая по популярности пара: Удаленная работа - Полная занятость (пока в ~ 4.5 раза меньше). Что также ожидаемо, ведь полная занятось это гарантия хорошего стабильного заработка, а удаленный формат работы становится в наше время нормальным и привлекает людей все больше.
* Анализ требуемого опыта работы показал, что наиболее актуальным является опыт от года до трех. Это ожидаемо, так так данный опыт позволяет говорить о наличии у соискателя необходимых навыков и в тоже время такого сотрудника легче приспособить к конкретной должности. Далее по популярности идут пожелания чуть большего опыта, что характерно для должностей более сложных профессий или рода деятельности. То, что отсутствие опыта находится не на последнем месте свидетельствует, что работодатель готов обучать прямо на месте в деятельности. И  таких предложений даже больше, чем предложений, где важен опыт работы более 6 лет. 


# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [ ]:
# текст запроса
query_5_1 = f'''(SELECT  
                     e.name "Название_работодателя", 
                     Count(v.id) "Количество_вакансий"
                 FROM public.employers e
                     JOIN public.vacancies v on v.employer_id = e.id
                 GROUP BY 1
                 ORDER BY 2 DESC
                 LIMIT 1
                 )
                 UNION 
                 (SELECT  
                     e.name "Название_работодателя", 
                     Count(v.id) "Количество_вакансий"
                 FROM public.employers e
                     JOIN public.vacancies v on v.employer_id = e.id
                 GROUP BY 1
                 ORDER BY 2 DESC
                 OFFSET 4 LIMIT 1
                 )
                 ORDER BY 2 DESC
             '''

In [ ]:
# результат запроса
df = pd.read_sql_query(query_5_1, connection)
df

In [ ]:
|index|Название_работодателя|Количество_вакансий|
|-----|---------------------|-------------------|
|0    |    Яндекс           |    1933           |
|1    |    Газпром нефть    |     331           |

2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [ ]:
# текст запроса
query_5_2 = f'''SELECT 
                    a.name as region, 
                    COUNT(DISTINCT e.id) employers_count, 
                    COUNT(v.id) vacancies_count
                FROM public.areas a 
                    LEFT JOIN public.employers e on e.area=a.id 
                    LEFT JOIN public.vacancies v on v.area_id=a.id
                WHERE v.id IS NULL
                GROUP BY a.name
                ORDER BY COUNT(DISTINCT e.id) DESC
                LIMIT 1
              '''

In [ ]:
df = pd.read_sql_query(query_5_2, connection)
df

In [ ]:
# результат запроса
|index|region|employers_count|vacancies_count|
|-----|------|---------------|---------------|
|  0  |Россия|      410      |      0        |

3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [ ]:
# текст запроса
query_5_3 = f'''SELECT  
                     e.name "Название_работодателя", 
                     Count(distinct v.area_id) "Количество_регионов"
                 FROM public.employers e
                     JOIN public.vacancies v on v.employer_id = e.id
                 GROUP BY e.name
                 ORDER BY 2 DESC
                 LIMIT 1
              '''

In [ ]:
df = pd.read_sql_query(query_5_3, connection)
df

In [ ]:
# результат запроса
|index|Название_работодателя|Количество_регионов|
|-----|---------------------|-------------------|
|  0  |       Яндекс        |       181         |

4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [ ]:
# текст запроса
query_5_4 = f'''SELECT  
                    Count(*) "Число_без_сфер_деятельности"
                FROM public.employers e
                    LEFT JOIN public.employers_industries em on em.employer_id = e.id
                WHERE employer_id IS NULL
             '''

In [ ]:
df = pd.read_sql_query(query_5_4, connection)
df

In [ ]:
# результат запроса
|index|Число_без_сфер_деятельности|
|-----|---------------------------|
|  0  |         8419              |

5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [ ]:
# текст запроса
query_5_5 = f'''SELECT  
                     e.name name_employer,
                     COUNT(*) count_industries
                 FROM public.employers e
                     LEFT JOIN public.employers_industries em ON em.employer_id = e.id
                 GROUP BY e.name
                 HAVING COUNT(*) = 4
                 ORDER BY 1
                 LIMIT 3
              '''

In [ ]:
df = pd.read_sql_query(query_5_5, connection)
df

<ipython-input-16-b4fd0715c68c>:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_5_5, connection)


,name_employer,count_industries
0,101 Интернет,4
1,21vek.by,4
2,2ГИС,4


In [ ]:
# результат запроса
|index|name_employer|count_industries|
|-----|-------------|----------------|
|  0  | 101 Интернет|   4            |
|  1  |  21vek.by   |   4            |
|  2  |  2ГИС       |   4            |

6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [ ]:
# текст запроса
query_5_6 = f'''SELECT  
                     COUNT(DISTINCT em.employer_id) dev_pro
                 FROM public.employers_industries em 
                     JOIN public.industries i ON em.industry_id = i.id
                 WHERE i.name = 'Разработка программного обеспечения'
             '''

In [ ]:
df = pd.read_sql_query(query_5_6, connection)
df

In [ ]:
# результат запроса
|index|dev_pro|
|-----|-------|
|  0  | 3553  |

7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [ ]:
# код для получения списка городов-милионников
import requests # Импортируем библиотеку requests
from bs4 import BeautifulSoup # Импортируем библиотеку BeautifulSoup
url = 'https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8'
text = requests.get(url) 
 
table_cities = BeautifulSoup(text.text, 'html.parser').find('span', id = 'Список_городов-миллионеров')
table_cities = table_cities.parent.next_sibling

while table_cities.name != 'table':
  table_cities = table_cities.next_sibling

new_table_cities = table_cities.find('tbody').find_all('tr')

big_city = []
for item in new_table_cities:
  trs = item.find_all('td')
  if len(trs) > 1:
    big_city.append(trs[1].text.strip().replace('[a]', ''))
print(f'Городов-миллионников всего: {len(big_city)}.\nЭто такие города, как:')
print(*big_city, sep='\n')

Городов-миллионников всего: 16.
Это такие города, как:
Москва
Санкт-Петербург
Новосибирск
Екатеринбург
Казань
Нижний Новгород
Красноярск
Челябинск
Самара
Уфа
Ростов-на-Дону
Омск
Краснодар
Воронеж
Пермь
Волгоград


In [ ]:
# текст запроса
big_city = "'" + "','".join(big_city) + "'"
company = "'Яндекс'"
query_5_7 = f'''(
                SELECT 
                    a.name as name_region, 
                    COUNT(*) employers_count 
                FROM public.areas a 
                    JOIN public.vacancies v ON v.area_id = a.id
                    JOIN public.employers e ON e.id = v.employer_id                     
                WHERE a.name IN ({big_city}) AND e.name = {company}
                GROUP BY a.name
                ORDER BY employers_count ASC
                )
                UNION
                (
                SELECT
                   'Total', 
                    COUNT(*) 
                FROM public.areas a
                    JOIN public.vacancies v ON v.area_id = a.id
                    JOIN public.employers e ON e.id = v.employer_id                     
                WHERE a.name IN ({big_city}) AND e.name = {company}
                )
                ORDER BY  employers_count 
              '''
df = pd.read_sql_query(query_5_7, connection)
df

In [ ]:
# результат запроса
|index|name\_region|employers\_count|
|-----|------------|----------------|
|  0  |   Омск     |     21         |
|  1  | Челябинск  |     22         |
|  2  | Красноярск |     23         |
|  3  | Волгоград  |     24         |
|  4  |   Пермь    |     25         |
|  5  |   Казань   |     25         |
|  6  |Ростов-на-Дону|   25         |
|  7  |    Уфа     |     26         |
|  8  |   Самара   |     26         |
|  9  |  Краснодар |     30         |
|  10 |  Воронеж   |     32         |
|  11 | Новосибирск|     35         |
|  12 |Нижний Новгород|  36         |
|  13 |Екатеринбург|     39         |
|  14 |Санкт-Петербург|  42         |
|  15 |   Москва   |     54         |
|  16 |   Total    |     485        |

***

In [ ]:
# выводы по анализу работодателей


* Лидером по предложению вакансий является компания Яндекс, которая предлагает в 4 раза больше вакансий, чем Ростелеком - компания, которая на втором месте. В топ входят довольно крупные "игроки": IT-компании, Банкинг, Телеком, Нефть, где идет постоянное активное развитие, достаточно высокая текучесть кадров, расширение бизнеса.
*  Среди регионов, в которых нет вакансий, по количеству работодателей в лидерах оказалась Россия. Это ожидаемо, тк Россия очень большая по протяженности с большим количеством городов_миллионников, с одной стороны, и плохоразвитой местностью на периферии. Небольшие компании или компании из небольших городов или регионов склонны указывать наиболее крупный регион, к которому относится. К тому же в России есть компании, которые предлагают вакансии за границей. 
* Наибольшее количество регионов затронуто  у Яндекса, Ростелекома, Спецремонта, это может быть связано с хорошо развитой сетью офисов по регионам.
* Анализ показал, чт окомпания Яндекс предлагает свои вакансии в каждом из городов-миллионников пропорционально чесленности населения региона

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [ ]:
# текст запроса
query_6_1 = f'''SELECT  
                     COUNT(*) data_vac
                 FROM public.vacancies 
                 WHERE lower(name) LIKE '%data%' OR lower(name) LIKE '%данн%'
             '''
df = pd.read_sql_query(query_6_1, connection)
df

In [ ]:
# результат запроса
|index|data_vac|
|-----|--------|
|  0  |  1771  |

2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [ ]:
# текст запроса
DS = '''(
        LOWER(v.name) LIKE '%data scientist%'
        OR LOWER(v.name) LIKE '%data science%'
        OR LOWER(v.name) LIKE '%исследователь данных%'
        OR LOWER(v.name) LIKE '%machine learning%'
        OR LOWER(v.name) LIKE '%машинн%обучен%'
        OR (UPPER(v.name) LIKE '%ML%' 
            AND UPPER(v.name) NOT LIKE '%HTML%')
        )'''

query_6_2 = f'''SELECT 
                    COUNT(*) cnt_jun_vac
                FROM public.vacancies v
                WHERE {DS}
                    AND (LOWER(name) LIKE '%junior%' 
                    OR experience = 'Нет опыта'
                    OR employment = 'Стажировка')
                '''
df = pd.read_sql_query(query_6_2, connection)
df

In [ ]:
# результат запроса
|index|cnt_jun_vac|
|-----|-----------|
|  0  |    51     |

3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [ ]:
# текст запроса
DS = '''(
        Lower(name) LIKE '%data scientist%'
        OR Lower(name) LIKE '%data science%'
        OR Lower(name) LIKE '%исследователь данных%'
        OR Lower(name) LIKE '%machine learning%'
        OR Lower(name) LIKE '%машинн%обучен%%'
        OR (name LIKE '%ML%'
            AND name NOT LIKE '%HTML%'
        )
)'''

query_6_3 = f'''SELECT 
                    COUNT(*) cnt_SQL_vac
                FROM public.vacancies v
                WHERE {DS}
                    AND (UPPER(v.key_skills) LIKE '%SQL%' 
                         OR LOWER(v.key_skills) = '%postgres%')
             '''
df = pd.read_sql_query(query_6_3, connection)
df             

In [ ]:
# результат запроса
|index|cnt_sql_vac|
|-----|-----------|
|  0  |    201    |

4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [ ]:
# текст запроса
DS = '''(
        Lower(name) LIKE '%data scientist%'
        OR Lower(name) LIKE '%data science%'
        OR Lower(name) LIKE '%исследователь данных%'
        OR Lower(name) LIKE '%machine learning%'
        OR Lower(name) LIKE '%машинн%обучен%%'
        OR (name LIKE '%ML%'
            AND name NOT LIKE '%HTML%'
        )
)'''

query_6_4 = f'''SELECT 
                    COUNT(*) cnt_py_vac
                FROM public.vacancies v
                WHERE {DS}
                    AND LOWER(v.key_skills) LIKE '%python%'
             '''
df = pd.read_sql_query(query_6_4, connection)
df 

In [ ]:
# результат запроса
|index|cnt_py_vac|
|-----|-----------|
|  0  |    351    |

5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [ ]:
# текст запроса
DS = '''(
        Lower(name) LIKE '%data scientist%'
        OR Lower(name) LIKE '%data science%'
        OR Lower(name) LIKE '%исследователь данных%'
        OR Lower(name) LIKE '%machine learning%'
        OR Lower(name) LIKE '%машинн%обучен%%'
        OR (name LIKE '%ML%'
            AND name NOT LIKE '%HTML%'
        )
)'''

query_6_5 = f'''SELECT 
                    ROUND(AVG(1 + length(key_skills) - length(REPLACE(key_skills, CHR(9), '' )))::numeric, 2) avg_ds
                FROM public.vacancies v
                WHERE {DS}
             '''
df = pd.read_sql_query(query_6_5, connection)
df 

In [ ]:
# результат запроса
|index|avg_ds|
|-----|------|
|  0  | 6.41 |

6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [ ]:
# текст запроса
DS = '''(
        Lower(name) LIKE '%data scientist%'
        OR Lower(name) LIKE '%data science%'
        OR Lower(name) LIKE '%исследователь данных%'
        OR Lower(name) LIKE '%machine learning%'
        OR Lower(name) LIKE '%машинн%обучен%%'
        OR (name LIKE '%ML%'
            AND name NOT LIKE '%HTML%'
        )
)'''

query_6_6 = f'''SELECT experience,
                       Round(Avg((Coalesce(salary_from, salary_to) + Coalesce(salary_to, salary_from)) / 2)) ds_avg_salary
                FROM vacancies
                WHERE (salary_to IS NOT NULL
                        OR salary_from IS NOT NULL
                       ) 
                       AND {DS}
                       AND experience = 'От 3 до 6 лет'
                GROUP BY 1
                '''

df = pd.read_sql_query(query_6_6, connection)
df 

In [ ]:
# результат запроса
|index| experience  |ds_avg_salary|
|-----|-------------|-------------|
|  0  |От 3 до 6 лет|  243115.0   |

***

In [ ]:
# выводы по предметному анализу

* Вакансии конкретно для *DataSience* занимают примерно 3.6% (1771 / 49197 * 100) от общего числа вакансий.
* Среди представленных вакансий конкретно для *DataSience* на позицию *Junior* приходится 2.9% (51 / 1771 * 100).
* Среди вакансий для *DataSience* в качестве ключевых навыков достаточно часто встречается SQL. Такой навык указан примено в 20% вакансий для *DataSience* (351 / 1771 * 100).
* Python в качестве ключевого навыка для *DataSience* указан примерно в 11% вакансий (201 / 1771 * 100).
* Можно заметить, что в целом для *DataSience* работодателя интересует наличие примерно 6 ключевых навыков у соикателя.
* Заработная плата с ростом опыта каждый раз увеличивается примерно в 2 раза. 

# Общий вывод по проекту

In [ ]:
# подведем итог исследования, обобщите выводы
# здесь можно (это будет плюсом) провести дополнительные исследования данных, сделать прогнозы, продумать варианты продолжения исследования


* Наиболее крупные регионы представлены большим числом работодателей, которые предлагают более высокие зарплаты. Это можно учитывать при составлении прогнозов на будущее. 
* Топовые работодатели имеют обширную сеть офисов и стараются охватить большое число регионов. Это связано с тем, что работодатели готовы работать с соискателем в месте его локации, если это хороший специалист, с одной стороны, и с тем, что достойных специалистов можно найти и на периферии.
* Профессия *DataSience* представлена отнюдь не так широко, но при этом имеет достойную оплату даже на позиции *Junior*
* Важно понимать, что одного языка программирования не достаточно для крупных игроков в данной сфере. Требуется параллельно развивать несколько ключевых навыков, чтобы иметь возможность конкурировать на рынке труда.
* Запросы, представленные ниже позволяют выявить ТОП-50 вакансий, связанных с ИТ, среди которых можно наблюдать таких лидеров, как  "Системный администратор" и "Программист 1С". Однако, далее можно увидеть, что при всей актуальности даных вакансий, зарплатные планки данных специалистов не столь высоки в сравнении, например, с тестировщиками или специалистами по информационной безопасности, как это можно увидеть из последнего запроса. 

In [7]:
query_7_1 = f'''
    SELECT name "Топ-50 вакансий", 
        COUNT(*) vacancy_count
    FROM vacancies
    GROUP BY 1
    ORDER BY 2 DESC
    LIMIT 50
'''
df = pd.read_sql_query(query_7_1, connection)
df 

<ipython-input-7-fa6ee1d6962c>:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_7_1, connection)


,Топ-50 вакансий,vacancy_count
0,Системный администратор,1761
1,Программист 1С,1153
2,Бизнес-аналитик,488
3,Специалист технической поддержки,457
4,Системный аналитик,432
5,Аналитик,404
6,Инженер-программист,389
7,Программист 1C,360
8,Frontend-разработчик,242
9,Инженер технической поддержки,230


In [9]:
query_7_2 = f'''(SELECT 'Системный администратор' object_description,
            experience,
            Round(Avg(
                    (Coalesce(salary_from, salary_to) + Coalesce(salary_to, salary_from)) / 2
                )) avg_salary
    FROM vacancies
    WHERE  (salary_to IS NOT NULL
            OR salary_from IS NOT NULL
           ) 
           AND (
        Lower(name) LIKE '%cистемный администратор%'
)  
GROUP BY 1,2)
UNION
(SELECT 'Тестировщик' object_description,
            experience,
            Round(Avg(
                    (Coalesce(salary_from, salary_to) + Coalesce(salary_to, salary_from)) / 2
                )) avg_salary
    FROM vacancies
    WHERE  (salary_to IS NOT NULL
            OR salary_from IS NOT NULL
           ) 
           AND (
        Lower(name) LIKE '%тестиров%'
        OR Lower(name) LIKE '%qa engin%'
        OR Lower(name) LIKE '%qa-специалист%'
       
)  
GROUP BY 1,2)
UNION
(SELECT 'Программист' object_description,
            experience,
            Round(Avg(
                    (Coalesce(salary_from, salary_to) + Coalesce(salary_to, salary_from)) / 2
                )) avg_salary
    FROM vacancies
    WHERE  (salary_to IS NOT NULL
            OR salary_from IS NOT NULL
           ) 
           AND (
        Lower(name) LIKE '%программист%'
        OR Lower(name) LIKE '%programmer%'
        OR Lower(name) LIKE '%программист 1С%'
       
)  
GROUP BY 1,2)
UNION
(SELECT 'Аналитик данных' object_description,
            experience,
            Round(Avg(
                    (Coalesce(salary_from, salary_to) + Coalesce(salary_to, salary_from)) / 2
                )) avg_salary
    FROM vacancies
    WHERE  (salary_to IS NOT NULL
            OR salary_from IS NOT NULL
           ) 
           AND (
        Lower(name) LIKE '%аналитик%'
        OR Lower(name) LIKE '%data analytics%'
        OR Lower(name) LIKE '%data analyst%'
)  
GROUP BY 1,2)
UNION
(SELECT 'Разработчик' object_description,
            experience,
            Round(Avg(
                    (Coalesce(salary_from, salary_to) + Coalesce(salary_to, salary_from)) / 2
                )) avg_salary
    FROM vacancies
    WHERE  (salary_to IS NOT NULL
            OR salary_from IS NOT NULL
           ) 
           AND (
        Lower(name) LIKE '%разработчик%'
        OR Lower(name) LIKE '%developer%'
        OR Lower(name) LIKE '%devops%'
)  
GROUP BY 1,2)
UNION 
(SELECT 'Специалист по информационной безопасности' object_description,
            experience,
            Round(Avg(
                    (Coalesce(salary_from, salary_to) + Coalesce(salary_to, salary_from)) / 2
                )) avg_salary
    FROM vacancies
    WHERE  (salary_to IS NOT NULL
            OR salary_from IS NOT NULL
           ) 
           AND (
        Lower(name) LIKE '%безопасност%'
        OR Lower(name) LIKE '%secops%'
        OR Lower(name) LIKE '%цифров%кримин%'
        OR Lower(name) LIKE '%этичн%хакер%'
        OR Lower(name) LIKE '%пентестер%'
        OR Lower(name) LIKE '%охотн%за ошибками%'
        OR Lower(name) LIKE '%иб-специалист%'
)  
GROUP BY 1,2)
ORDER BY avg_salary'''

df = pd.read_sql_query(query_7_2, connection)
df 


<ipython-input-9-2a4bf3aacde6>:102: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_7_2, connection)


,object_description,experience,avg_salary
0,Системный администратор,От 1 года до 3 лет,35069.0
1,Тестировщик,Нет опыта,44151.0
2,Специалист по информационной безопасности,Нет опыта,44955.0
3,Программист,Нет опыта,47856.0
4,Системный администратор,От 3 до 6 лет,48000.0
5,Аналитик данных,Нет опыта,48338.0
6,Специалист по информационной безопасности,От 1 года до 3 лет,60939.0
7,Разработчик,Нет опыта,71473.0
8,Тестировщик,От 1 года до 3 лет,79618.0
9,Аналитик данных,От 1 года до 3 лет,81290.0
